In [1]:
import pandas as pd
import json
import time
import os
import dill
from docembedder.bert import BERTEmbedder
from docembedder.analysis import DOCSimilarity
from docembedder.preparation import DOCPreparation
from docembedder.analysis import DOCSimilarity

## Data preparation

In [2]:
%%time
# # Read and preprocess Patent's documents
# patent_document = DOCPreparation()
# patent_df = patent_document.read_patents()
# patent_df.shape

# # Do some preprocessing here

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


## Read patent's dataset

In [3]:
%%time
patent_df = pd.read_csv('data/patents_concatenated.csv')
patent_df.shape

CPU times: user 29 s, sys: 4.07 s, total: 33 s
Wall time: 35.2 s


(426089, 3)

In [4]:
# Removing null values
patent_df.dropna(inplace=True)
patent_df.reset_index(drop=True, inplace=True)
print('shape:', patent_df.shape)

shape: (423057, 3)


#### Dataset overview

In [5]:
patent_df.index

RangeIndex(start=0, stop=423057, step=1)

In [6]:
patent_df.groupby('year').groups.keys()

dict_keys([0, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1946, 1947, 1948, 1949, 1950, 1953])

In [7]:
patent_df.groupby(['year']).agg('count')

,patent,contents
year,,
0,8,8
1838,58,58
1839,391,391
1840,447,447
1841,481,481
1842,476,476
1843,481,481
1844,461,461
1845,467,467


In [8]:
# Remove year 0 and year 1953
patent = patent_df[(patent_df['year']!=0) & (patent_df['year']!=1953)]
patent.shape

(423048, 3)

## Embeddings generation

In [9]:
# patent_embedder = BERTEmbedder()

# patent_embedder.fit(patent_df['contents'])

## Similarity analysis

In [10]:
patent_analyser = DOCSimilarity.from_dill()
patent_analyser.compute_similarity()
patent_analyser.df_patents_embeddings


,index,patent,contents,year,embeddings,novelty,impact
0,84750,1046,is formed by being beaten closely about the en...,1838,"[-0.09769521653652191, -1.2423615455627441, 3....",NaN,NaN
1,84441,1017,"25, 1838. 55 60 50 surface of the cylinder as ...",1838,"[-0.4417773187160492, -0.9246848821640015, 4.0...",NaN,NaN
2,84355,1009,"all whom, it may concern be it known that i, t...",1838,"[-0.30841735005378723, -1.3443700075149536, 3....",NaN,NaN
3,84267,1001,"1838, pgrefis. foto-thographer, washington, un...",1838,"[-0.3922632038593292, -0.01046842336654663, 3....",NaN,NaN
4,84664,1038,"all whom it may concern: be it known that i, s...",1838,"[-0.6282005310058594, -0.8042935729026794, 4.0...",NaN,NaN
...,...,...,...,...,...,...,...
515,384749,2499851,"mar. 1950 ,499,851 united states patent office...",1950,"[-0.690518856048584, -0.9980224370956421, 3.43...",0.061658,0.994850
516,32125,2517446,nov. 1945 sheets-sheet 777tee wawawwww a a as ...,1950,"[0.3316209316253662, -0.7649397850036621, 2.87...",0.036216,1.000385
517,367521,2504159,",504,159 anemal feed and method of mproving th...",1950,"[0.2976904511451721, -0.8043850660324097, 2.81...",0.049836,0.996173
518,367482,2504120,",504,120 pasticzed polywny, compostions harry ...",1950,"[0.4777756333351135, -1.2891647815704346, 2.86...",0.049129,0.995120


In [25]:
import pandas as pd
df = pd.read_csv('data/patents_concatenated.csv')
df.shape

(282997, 3)